In [2]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [7]:
import csv
import itertools
from tabulate import tabulate

class ScheduleGenerator:
    def __init__(self, courses, faculty):
        self.courses = courses
        self.faculty = faculty
        self.schedule = {}
        self.days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
        self.times = [
            '7:00-8:40AM', '9:00-10:40AM', '11:00-12:40PM',
            '1:20-3:00PM', '3:30-5:10PM', '5:30-7:10PM'
        ]

    def generate_schedule(self):
        all_slots = list(itertools.product(self.days, self.times))
        assigned_courses = set()

        for faculty, courses in self.faculty.items():
            for course in courses:
                if course in assigned_courses:
                    continue

                for slot in all_slots:
                    day, time = slot
                    if self.is_slot_available(course, faculty, day, time):
                        self.schedule[course] = (faculty, day, time)
                        assigned_courses.add(course)
                        break

        remaining_courses = set(self.courses) - assigned_courses
        self.schedule_remaining_courses(remaining_courses)

    def is_slot_available(self, course, faculty, day, time):
        for assigned_course, (assigned_faculty, assigned_day, assigned_time) in self.schedule.items():
            if assigned_faculty == faculty and (day == assigned_day and time == assigned_time):
                return False
        return True

    def schedule_remaining_courses(self, remaining_courses):
        all_slots = list(itertools.product(self.days, self.times))
        for course in remaining_courses:
            for slot in all_slots:
                day, time = slot
                if self.is_slot_available(course, self.faculty[course], day, time):
                    self.schedule[course] = (self.faculty[course], day, time)
                    break

    def get_schedule_table(self, selected_courses):
        table_data = []
        for course in selected_courses:
            if course in self.schedule:
                faculty, day, time = self.schedule[course]
                table_data.append([course, faculty, day, time])
        return table_data

if __name__ == "__main__":
    courses = []
    faculty = {}

    # Read data from CSV file and populate courses and faculty dictionaries
    with open("ProjectSchedule.csv", "r") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            courses.append(row["COURSE NO"])
            faculty[row["COURSE NO"]] = row["FACULTY"]

    schedule_generator = ScheduleGenerator(courses, faculty)

    max_attempts = 5
    selected_courses = []
    for attempt in range(max_attempts):
        user_input = input("Enter the course number (attempt {} of {}): ".format(attempt + 1, max_attempts))
        if user_input in courses and user_input not in selected_courses:
            conflicting_courses = set(selected_courses) & {course for course, (_, _, time) in schedule_generator.schedule.items() if time == schedule_generator.schedule[user_input][2]}
            if not conflicting_courses:
                selected_courses.append(user_input)
                if len(selected_courses) >= 5:
                    break
            else:
                print("Time conflict with courses: {}".format(', '.join(conflicting_courses)))
        else:
            print("Invalid course number or course already selected. Try again.")

    if len(selected_courses) > 0:
        schedule_generator.generate_schedule()
        table_data = schedule_generator.get_schedule_table(selected_courses)
        print("\nSelected Courses:")
        print(tabulate(table_data, headers=["Course", "Faculty", "Day", "Time"], tablefmt="grid"))
    else:
        print("No courses selected.")

Enter the course number (attempt 1 of 5): APT1050
Enter the course number (attempt 2 of 5): APP4025
Enter the course number (attempt 3 of 5): FIN4030
Enter the course number (attempt 4 of 5): MKT4030
Enter the course number (attempt 5 of 5): APT3010

Selected Courses:
+----------+------------+-------+--------------+
| Course   | Faculty    | Day   | Time         |
+==========+============+=======+==============+
| APT1050  | Wamuyu, P  | Mon   | 9:00-10:40AM |
+----------+------------+-------+--------------+
| FIN4030  | Wainaina S | Mon   | 7:00-8:40AM  |
+----------+------------+-------+--------------+
| MKT4030  | Maisori, B | Mon   | 7:00-8:40AM  |
+----------+------------+-------+--------------+
